In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [ ]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)

In [ ]:
conv_base.trainable = True

set_trainable = False

for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

for layer in conv_base.layers:
  print(layer.name,layer.trainable)

In [ ]:
conv_base.summary()

In [ ]:

model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [1]:
# conv_base.trainable = False
# For Data Augmentation
from keras.preprocessing.image import ImageDataGenerator
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.3,
        horizontal_flip=True
        )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'Train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = test_datagen.flow_from_directory(
        'Test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 940 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [ ]:
# from tensorflow import keras
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])


In [ ]:
history = model.fit_generator(train_generator,
                              epochs=20,
                              validation_data=validation_generator)

In [ ]:
model.save('Fog_NonFoggy2.h5')

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# 1 -- Clean
# 0 -- Fog
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

model = tf.keras.models.load_model('A:\MINI_PROJECT_1\Fog_NonFoggy2.h5')

# test_image = cv2.imread('FOG_Test_Input.jpg')
test_image = cv2.imread(r"A:\MINI_PROJECT_1\OUTPUT_TEST\CLEAN1.jpeg")
#plt.imshow(test_image)
# test_image.shape
test_image = cv2.resize(test_image,(150,150))
test_image.shape
test_image = test_image.reshape(1,150,150,3)


prob = model.predict(test_image)
print(prob)
# print(prob[0])
# print(prob[1])
# if (prob > 0.5 ):
#     print("Clean")
# else:
#     print("Foggy")

In [ ]:
import cv2
import tensorflow as tf
model = tf.keras.models.load_model('Fog_NonFoggy2.h5')

# Function to preprocess an image
def preprocess_image(img):
    img = cv2.resize(img, (150,150))
    img = img.reshape(1, 150,150, 3)
    return img

# Path to the video file
video_path = 'A:\MINI_PROJECT_1\OUTPUT_TEST\clean-v1-ip.mp4'

# Open the webcam
cap = cv2.VideoCapture(video_path)
# Load the model

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.resize(frame,(500,500))
    # Display the resulting frame
    cv2.imshow('Original Frame', frame)

    # Preprocess the frame for prediction
    processed_frame = preprocess_image(frame)

    # Make prediction
    prediction = model.predict(processed_frame)

    # Print or display the prediction (modify this part based on your model's output)
    if prediction > 0.5:
        print("Clean")
        print(prediction)
    else:
        print("Foggy")
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and destroy all windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
model.summary()